In [1]:
%matplotlib notebook

In [1]:
import pyspark
sc = pyspark.sql.SparkSession.Builder().getOrCreate()

In [2]:
import json
import numpy as np
import pandas as pd
import seaborn as sn

import matplotlib.pyplot as plt
from IPython.display import display, HTML
import math
import time
import matplotlib.animation as animation
from datetime import datetime, timedelta
import pyspark.sql.functions as F

from utils.emissions_normalizer import EmissionsNormalizer
from utils.knn_predictor import KnnPredictor
from utils.smoother import Smoother
from utils.random_predictor import RandomPredictor

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [3]:
ANTENNAS_POSITION = [(464259.981343845,6110331.85100085),(463512.015195402,6111004.324434620),(464443.295130103,6111377.26171875),(464629.562194595,6111105.34734669)]
ANTENNAS_NAMES = ["RC1", "RC2", "D1", "D2"] 
X_0 = 462385.503783397
Y_0 = 6109042.35153865

for i in range(4):
    ANTENNAS_POSITION[i] = (ANTENNAS_POSITION[i][0]-X_0, ANTENNAS_POSITION[i][1]-Y_0)

GRID = Polygon(ANTENNAS_POSITION)

In [4]:
#GRID = Polygon([(1750,1600), (1750,2000), (2200, 2000), (2200, 1600)])
#GRID = Polygon([(1450,1800),(1450,2000),(2200,2000),(2200,1800)])



FEMALES = [11, 17, 20, 24, 26, 28, 30, 33, 34]
MALES = [10, 14, 21, 22, 23, 25, 27, 29, 31, 32]

## Entreno el modelo que vamos a usar para predecir

In [5]:
points_recep = sc.read.json('datos/train-test-by-emission.jsonlines/').rdd

In [6]:
normalizer = EmissionsNormalizer()
data = normalizer.normalize(points_recep)
regre_data, regre_target = normalizer.get_regression_dataframes(data)

In [7]:
predictor = KnnPredictor()
predictor.fit(regre_data, regre_target)

### Levanto el chekpoint en pandas

In [8]:
birds_data_complete = pd.read_parquet('tmp/checkpoint-cog.parquet')
birds_data = birds_data_complete.drop(columns=['tag_id','timestamp'])

### Obtengo las predicciones

In [22]:
predictions = predictor.predict(birds_data)
predictions = pd.concat([pd.DataFrame(predictions), pd.DataFrame(birds_data_complete)], axis=1).values

## Analizo coocurrencia en regiones uniformes

### Calculo en que region cayo la prediccion

In [23]:
X_REGION_SIZE = 300
Y_REGION_SIZE = 300


def set_prediction_in_regions_classification(predictions):
    grid_predictions = []
    for i in range(len(predictions)):
        prediction = predictions[i]
        point = Point(prediction[0], prediction[1])

        region = '{}-{}'.format(int(prediction[0]/X_REGION_SIZE), int(prediction[1]/Y_REGION_SIZE))

        grid_pred = np.insert(prediction, 8,region, axis=0)
        grid_predictions.append(grid_pred)
    return pd.DataFrame(grid_predictions, columns=['x', 'y', 'recep_0', 'recep_1', 'recep_2', 'recep_3', 'tag', 'time', 'region'])


### Obtengo el tiempo que cada posible pareja estuvo junta

In [24]:
def get_bird_timestamp_in_regions(bird_data, delta_time_in_sec):
    """
    delta_time_in_sec es la cantidad de segundos que decimos que el pajaro se encuentra en un mismo punto dada una prediccion
    Por default, el valor es 5, ya que las emisiones son cada 5 segundos.
    """
    timestamp_in_grid = []
    for row in zip(bird_data.time, bird_data.region):
        real_time = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
        for i in range(-delta_time_in_sec, delta_time_in_sec+1):
            res=real_time+timedelta(seconds=i)
            timestamp_in_grid.append((row[1], datetime.strftime(res, '%Y-%m-%d %H:%M:%S')))

    timestamp_in_grid = list(set(timestamp_in_grid))
    return pd.DataFrame(timestamp_in_grid, columns=['region','time'])


def get_matches_per_bird_in_regions(grid_predictions, delta_time):
    male_birds_in_grid = grid_predictions[(grid_predictions.tag.isin(MALES)) & (grid_predictions.region.notnull())]

    matches_per_bird_near_antenna = {}
    for female_bird in FEMALES:
        female_bird_in_grid = grid_predictions[(grid_predictions['tag'] == female_bird) & (grid_predictions.region.notnull())]
        timestamp_in_grid_pd = get_bird_timestamp_in_regions(female_bird_in_grid, delta_time)

        male_in_grid_with_female = male_birds_in_grid.join(timestamp_in_grid_pd.set_index(['region','time']), on=['region','time'], how='inner')

        female_matches = {}
        for index, row in male_in_grid_with_female.iterrows():
            tag = row['tag']
            current_matches = female_matches.get(tag, 0)
            female_matches[tag] = current_matches+1

        matches_per_bird_near_antenna[female_bird] = female_matches
    return matches_per_bird_near_antenna



In [27]:
predictions = set_prediction_in_regions_classification(predictions)
matches_per_bird = get_matches_per_bird_in_regions(predictions, 5)


matrix_res = [[]] * len(FEMALES)
for i in range(len(FEMALES)):
    matrix_res[i] = []
    for m in MALES:
        matrix_res[i].append(matches_per_bird.get(FEMALES[i]).get(m, 0))



In [29]:
matrix_res_pd = pd.DataFrame(matrix_res)
matrix_res_pd.columns = MALES
matrix_res_pd.rename(index={i:FEMALES[i] for i in range(len(FEMALES))}, inplace=True)
matrix_res_pd.to_csv('prettier/matrix_coocurrence.csv')

In [33]:
matrix_res_pd

,10,14,21,22,23,25,27,29,31,32
11,804,18,153,0,26,4,374,242,69,115
17,99,608,719,388,87,213,504,562,1553,750
20,295,646,6058,277,179,97,764,783,1208,1704
24,228,311,758,243,1174,19,862,614,1387,1374
26,14,215,170,222,32,807,28,114,64,54
28,99,414,280,150,35,21,72,76,293,80
30,92,611,361,330,115,166,212,209,3317,476
33,123,159,1011,114,171,27,675,1120,921,9395
34,183,218,486,120,106,56,930,5695,168,861


In [5]:
# to delete

In [6]:
areas = [[(1400.0, 730.0),
  (1152.7043433578376, 1125.7557782063323),
  (1604.9966613079316, 1408.3793857973747),
  (1852.292317950094, 1012.6236075910426)],
 [(1152.7043433578376, 1125.7557782063323),
  (905.4086867156753, 1521.5115564126645),
  (1357.7010046657692, 1804.135164003707),
  (1604.9966613079314, 1408.3793857973749)],
 [(905.4086867156752, 1521.5115564126645),
  (658.113030073513, 1917.2673346189968),
  (1110.4053480236068, 2199.890942210039),
  (1357.7010046657692, 1804.1351640037071)],
 [(1852.292317950094, 1012.6236075910426),
  (1604.9966613079316, 1408.3793857973747),
  (2057.2889792580254, 1691.0029933884175),
  (2304.584635900188, 1295.2472151820853)],
 [(1604.9966613079316, 1408.3793857973749),
  (1357.7010046657692, 1804.135164003707),
  (1809.9933226158632, 2086.7587715947498),
  (2057.2889792580254, 1691.0029933884175)],
 [(1357.7010046657692, 1804.1351640037071),
  (1110.4053480236068, 2199.890942210039),
  (1562.6976659737009, 2482.514549801082),
  (1809.9933226158632, 2086.7587715947498)],
 [(2304.584635900188, 1295.2472151820853),
  (2057.288979258026, 1691.0029933884175),
  (2509.5812972081194, 1973.62660097946),
  (2756.876953850282, 1577.8708227731279)],
 [(2057.2889792580254, 1691.0029933884175),
  (1809.993322615863, 2086.7587715947498),
  (2262.285640565957, 2369.382379185792),
  (2509.5812972081194, 1973.6266009794601)],
 [(1809.9933226158632, 2086.7587715947498),
  (1562.6976659737009, 2482.514549801082),
  (2014.9899839237949, 2765.1381573921244),
  (2262.2856405659572, 2369.382379185792)]]

In [7]:
map_areas = []
for area in areas:
    map_areas.append(Polygon(area))


In [14]:
map_areas[0].area

248888.88888888893

In [17]:
map_areas[0].length

2000.0000000000005